In [30]:
""" Página do Clube (1/4) """

from bs4 import BeautifulSoup
import requests
import re

url_benfica = "https://www.uefa.com/uefachampionsleague/clubs/50147--benfica/"
result = requests.get(url_benfica)

doc_benfica = BeautifulSoup(result.text, "html.parser")

In [32]:
""" Página do Clube (2/4) """

# find the div that contains the required stats
key_stats_tag = doc_benfica.find('div', class_="stats-module")

# list all divs that contain individual stats
stat_tags = key_stats_tag.find_all('div', class_="stats-module__single-stat--number")

# list all divs that contain individual stats with yellow and red cards
#stat_tags = key_stats_tag.find_all('div', class_=re.compile("stats-module__single-stat--number*"))

print("# Number of stat elements: ", len(stat_tags))
print("")

print("# First stat element:")
print(stat_tags[0].prettify())

# Number of stat elements:  9

# First stat element:
<div class="stats-module__single-stat stats-module__single-stat--number stats-module__single-stat--">
 <pk-num-stat-item class="" variant="secondary">
  <div slot="stat-value">
   4
  </div>
  <div slot="stat-label">
   Goals
  </div>
  <div slot="stat-second-label">
   2 avg. per match
  </div>
 </pk-num-stat-item>
</div>



In [33]:
""" Página do Clube (3/4) """

# Extract required data point
i = 0

# method 1
stat_tag = stat_tags[i].find_all('div')
stat_name = stat_tag[1].string
stat_value = stat_tag[0].string
print("Method 1:")
print("# stat name: ", stat_name)
print("# stat value: ", stat_value)
print("")

# method 2
stat_name = stat_tags[i].find(slot="stat-label").string
stat_value = stat_tags[i].find(slot="stat-value").string
print("Method 2:")
print("# stat name: ", stat_name)
print("# stat value: ", stat_value)

# print available stats
print("")
print("# available stats:")
print([stat.find(slot="stat-label").string for stat in stat_tags])

Method 1:
# stat name:  Goals
# stat value:  4

Method 2:
# stat name:  Goals
# stat value:  4

# available stats:
['Goals', 'Goals conceded', 'Possession (%)', 'Passing accuracy (%)', 'Balls recovered', 'Tackles won', 'Clean sheets', 'Saves', 'Distance covered (km)']


In [34]:
""" Página do Clube (4/4) """

key_stats_tag = doc_benfica.find('div', class_="stats-module")
stat_tags = key_stats_tag.find_all('div', class_=re.compile("stats-module__single-stat--number*"))

# loop through all stats and store the structered data
stats = {}

for stat_tag in stat_tags:
    name = stat_tag.find(slot="stat-label").string
    value = stat_tag.find(slot="stat-value").string

    # Think ahead and convert data into correct data type
    if "%" in name: value = float(value[:-1])
    elif name == "Distance covered (km)": value = float(value)
    else: value = int(value)

    stats[name] = value

for name, value in stats.items():print(f'{name}:  {value}')

Goals:  4
Goals conceded:  1
Possession (%):  54.0
Passing accuracy (%):  85.5
Balls recovered:  98
Tackles won:  5
Clean sheets:  1
Saves:  4
Distance covered (km):  239.83
Yellow cards:  4
Red cards:  0


In [35]:
""" Página com lista de clubes (1/4) """

url_clubs = "https://www.uefa.com/uefachampionsleague/clubs/"
result = requests.get(url_clubs)

doc_clubs = BeautifulSoup(result.text, "html.parser")

group_stage_tag = doc_clubs.find('div', class_="teams-overview_teams-wrapper")
teams_tag = group_stage_tag.find_all('div', class_="team team-is-club", limit=5)

print(teams_tag[0].prettify())



<div class="team team-is-club">
 <a class="team-wrap" href="/uefachampionsleague/clubs/50143--ajax/" title="Ajax">
  <pk-identifier class="team-wrap" type="vertical">
   <span slot="prefix">
    <pk-badge alt="Ajax" badge-title="Ajax" fallback-image="club-generic-badge" src="https://img.uefa.com/imgml/TP/teams/logos/70x70/50143.png">
    </pk-badge>
   </span>
   <span slot="primary">
    Ajax
   </span>
   <span class="team-name__country-code" slot="secondary">
    (NED)
   </span>
  </pk-identifier>
 </a>
</div>



In [36]:
""" Página com lista de clubes (2/4) """

print(teams_tag[0].a['title'])
print(teams_tag[0].a['href'])
print("")

print("# Full url:")
print("https://www.uefa.com" + teams_tag[0].a['href'])


Ajax
/uefachampionsleague/clubs/50143--ajax/

# Full url:
https://www.uefa.com/uefachampionsleague/clubs/50143--ajax/


In [37]:
""" Página com lista de clubes (3/4) """

# estrutura de dados
# [{ 
#   "name": "team1"
#   "url": "www.exemplo.com",
#   "stat1": 3,    
#   "stat2": 3,    
#   (...)    
#  },
#  { (...) }
# ]

teams = []

for team_tag in teams_tag:
    team_name = team_tag.a['title']
    team_url = team_tag.a['href']
    team_url = "https://www.uefa.com" + team_url
    
    teams.append({"name": team_name, "url": team_url})

print(f'Found {len(teams)} teams')
for team in teams: print(f"{team['name']}: {team['url']}")


Found 5 teams
Ajax: https://www.uefa.com/uefachampionsleague/clubs/50143--ajax/
Atlético: https://www.uefa.com/uefachampionsleague/clubs/50124--atletico/
Barcelona: https://www.uefa.com/uefachampionsleague/clubs/50080--barcelona/
Bayern: https://www.uefa.com/uefachampionsleague/clubs/50037--bayern/
Benfica: https://www.uefa.com/uefachampionsleague/clubs/50147--benfica/


In [38]:
""" Página com lista de clubes (4/4) """


for team in teams:
    print(f"Fetching {team['name']} data...")
    result = requests.get(team['url'])

    doc_club = BeautifulSoup(result.text, "html.parser") # request html and create bs4 instance

    key_stats_tag = doc_club.find('div', class_="stats-module")
    stat_tags = key_stats_tag.find_all('div', class_=re.compile("stats-module__single-stat--number.*"))

    for stat_tag in stat_tags:
        stat_name = stat_tag.find_all('div')[1].string
        stat_value = stat_tag.find_all('div')[0].string

        if "%" in stat_name: stat_value = float(stat_value[:-1])
        elif stat_name == "Distance covered (km)": stat_value = float(stat_value)
        else: stat_value = int(stat_value)

        team[stat_name] = stat_value

print(f"___________________________________")   
print(f"Fetched data from {len(teams)} teams")
print("")
print("### All fetched data:")
for team in teams:
    print(team)
    print("")


Fetching Ajax data...
Fetching Atlético data...
Fetching Barcelona data...
Fetching Bayern data...
Fetching Benfica data...
___________________________________
Fetched data from 5 teams

### All fetched data:
{'name': 'Ajax', 'url': 'https://www.uefa.com/uefachampionsleague/clubs/50143--ajax/', 'Goals': 5, 'Goals conceded': 2, 'Possession (%)': 57.5, 'Passing accuracy (%)': 87.0, 'Balls recovered': 70, 'Tackles won': 16, 'Clean sheets': 1, 'Saves': 9, 'Distance covered (km)': 230.86, 'Yellow cards': 2, 'Red cards': 0}

{'name': 'Atlético', 'url': 'https://www.uefa.com/uefachampionsleague/clubs/50124--atletico/', 'Goals': 2, 'Goals conceded': 3, 'Possession (%)': 52.0, 'Passing accuracy (%)': 85.0, 'Balls recovered': 65, 'Tackles won': 4, 'Clean sheets': 0, 'Saves': 5, 'Distance covered (km)': 236.8, 'Yellow cards': 2, 'Red cards': 0}

{'name': 'Barcelona', 'url': 'https://www.uefa.com/uefachampionsleague/clubs/50080--barcelona/', 'Goals': 5, 'Goals conceded': 3, 'Possession (%)': 63.0,

In [39]:
""" Gravar os dados para processamento """

import pandas as pd

data = pd.DataFrame(teams)
data.head()


,name,url,Goals,Goals conceded,Possession (%),Passing accuracy (%),Balls recovered,Tackles won,Clean sheets,Saves,Distance covered (km),Yellow cards,Red cards
0,Ajax,https://www.uefa.com/uefachampionsleague/clubs...,5,2,57.5,87.0,70,16,1,9,230.86,2,0
1,Atlético,https://www.uefa.com/uefachampionsleague/clubs...,2,3,52.0,85.0,65,4,0,5,236.80,2,0
2,Barcelona,https://www.uefa.com/uefachampionsleague/clubs...,5,3,63.0,90.0,81,8,0,2,235.41,1,0
3,Bayern,https://www.uefa.com/uefachampionsleague/clubs...,4,0,49.0,86.5,87,13,2,6,237.17,3,0
4,Benfica,https://www.uefa.com/uefachampionsleague/clubs...,4,1,54.0,85.5,98,5,1,4,239.83,4,0
